#dataset

In [2]:
# Install necessary packages
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-vosjh3gx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vosjh3gx
  Resolved https://github.com/huggingface/transformers.git to commit f26e4073707189c93915227779a4f6ea3c40d43b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9050615 sha256=aea052567481c338b42341fc5c638ae510c2816cee5d3ba16d651df2b58bb25e
  Stored in directory: /tmp/pip-ephem-wheel-cache-lzhjxxss/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface

In [3]:
# Import required libraries
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf
from tensorflow import keras

In [4]:
# Load the SQuAD dataset
datasets = load_dataset("rajpurkar/squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
# Convert train and validation datasets to DataFrames
Train = pd.DataFrame(datasets['train'])
Val = pd.DataFrame(datasets['validation'])

In [6]:
# Load the tokenizer
model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [7]:
# Set maximum sequence length and stride for tokenization
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The authorized overlap between two parts of the context when splitting


In [8]:

# Define a function to prepare training features
def prepare_train_features(examples):
  # Tokenize examples with truncation and padding, keeping overflows
  tokenized_examples = tokenizer(
      examples["question"],
      examples["context"],
      truncation="only_second",
      max_length=max_length,
      stride=doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

# Map from feature to example and compute start/end positions
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples["start_positions"] = []
  tokenized_examples["end_positions"] = []

  for i, offsets in enumerate(offset_mapping):
      input_ids = tokenized_examples["input_ids"][i]
      cls_index = input_ids.index(tokenizer.cls_token_id)

      sequence_ids = tokenized_examples.sequence_ids(i)

      sample_index = sample_mapping[i]
      answers = examples["answers"][sample_index]

      if len(answers["answer_start"]) == 0:
          tokenized_examples["start_positions"].append(cls_index)
          tokenized_examples["end_positions"].append(cls_index)
      else:
          start_char = answers["answer_start"][0]
          end_char = start_char + len(answers["text"][0])

          token_start_index = 0
          while sequence_ids[token_start_index] != 1:
              token_start_index += 1

          token_end_index = len(input_ids) - 1
          while sequence_ids[token_end_index] != 1:
              token_end_index -= 1

          if not (
              offsets[token_start_index][0] <= start_char
              and offsets[token_end_index][1] >= end_char
          ):
              tokenized_examples["start_positions"].append(cls_index)
              tokenized_examples["end_positions"].append(cls_index)
          else:
              while (
                  token_start_index < len(offsets)
                  and offsets[token_start_index][0] <= start_char
              ):
                  token_start_index += 1
              tokenized_examples["start_positions"].append(token_start_index - 1)
              while offsets[token_end_index][1] >= end_char:
                  token_end_index -= 1
              tokenized_examples["end_positions"].append(token_end_index + 1)

  return tokenized_examples

In [9]:
# Map training examples to tokenized features
tokenized_datasets = datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets["train"].column_names,
    num_proc=3,
)


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=3):   0%|          | 0/87599 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=3):   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
# Load tokenized datasets for training and validation
train_set = tokenized_datasets["train"].with_format("numpy")[:]
validation_set = tokenized_datasets["validation"].with_format("numpy")[:]


In [11]:
# Instantiate the question-answering model
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [12]:
# Configure optimizer
optimizer = keras.optimizers.Adam(learning_rate=5e-5)


In [13]:
# Optionally uncomment the next line for float16 training
# keras.mixed_precision.set_global_policy("mixed_float16")

# Compile the model
model.compile(optimizer=optimizer)


In [ ]:
# Train the model
model.fit(train_set, validation_data=validation_set, epochs=1)


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 376/2773 [===>..........................] - ETA: 53:13 - loss: 2.5653

In [ ]:
# Example context and question for inference
context = """Artificial intelligence (AI) is a branch of computer science that aims to create machines capable of intelligent behavior.
AI techniques have become an essential part of the technology industry, helping to solve many challenging problems in various domains.
Machine learning, a subset of AI, focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data.
Deep learning, a subfield of machine learning, involves the use of neural networks with many layers to analyze and learn from large amounts of data.
These technologies are driving innovations in areas such as natural language processing, computer vision, robotics, and autonomous vehicles."""
question = "What is artificial intelligence?"

In [ ]:
# Tokenize input for inference
inputs = tokenizer([context], [question], return_tensors="np")

# Perform inference
outputs = model(inputs)
start_position = tf.argmax(outputs.start_logits, axis=1)
end_position = tf.argmax(outputs.end_logits, axis=1)


In [ ]:
# Retrieve answer tokens
answer = inputs["input_ids"][0, int(start_position) : int(end_position) + 1]

# Decode answer tokens
print(tokenizer.decode(answer))
